In [ ]:
%%time

import time

from osgeo import ogr
from osgeo import osr

from qgis.core import *
from qgis.gui import *
from qgis.analysis import *

from qgis.PyQt.QtCore import *
from qgis.PyQt.QtGui import *

import datetime 

start = time.time()



'''
    Dijkstra:
        Este código crea rutas sintéticas que podría haber tomado un usuario al conocer un origen y un destino. En este caso, le estamos proporcionando las calles
        que conforman a la estructura vial de la ciudad de México, para que con QGIS podamos crear una red cuyos nodos sean cada intersección de cada calle, 
        avenida, eje vial, etc. Con esto, le damos al algortimo dos puntos, un origen y un destina y, con el algoritmo de Dijstra se busca la ruta más corta entre 
        el punto de origen y destino. Esto lo podemos replicar para buscar la ruta más corta en el STC, MB, etc.

        Con dichas rutas trabajaremos con nuestros modelos predictivos y compararémos los resultados con las rutas reales obtenidas por los usuarios.
'''


data = pd.read_csv('Datos_finales.csv') #obtenidos mediante la encuesta de movilidad


#En estas columnas tenemos los horarios de inicio y término de los viajes, algunos no marcan un valor y se le asignó el 99 por lo que hay que cambiarlo
data['p5_9_1'] = data['p5_9_1'].replace([99], 0)
data['p5_9_2'] = data['p5_9_2'].replace([99], 24)
data['p5_10_1'] = data['p5_10_1'].replace([99], 0)
data['p5_10_2'] = data['p5_10_2'].replace([99], 0)

#Los revolvemos solo porque están ordenados por alcaldías.

shuffled_data = data.sample(frac = 1,random_state = 42).reset_index()


# A los valores que cambiamos en los horarios los convertimos a la hora que corresponde y los guardamos en una lista para después agregarlos al DF

initial_time = []

for i in range(100000,100400):
    
    time = datetime.time(shuffled_data['p5_9_1'][i],shuffled_data['p5_9_2'][i])
    initial_time.append(str(time))


end_time = []

for i in range(len(shuffled_data)):
    
    time = datetime.time(shuffled_data['p5_10_1'][i],shuffled_data['p5_10_2'][i])
    end_time.append(str(time))
    
shuffled_data['initial_time'] = initial_time
shuffled_data['end_time'] = end_time

shuffled_data.drop(["p5_9_1", "p5_9_2","p5_10_1", "p5_10_2" ], axis = 1, inplace=True)

# cambiamos el sistema de coordenadas para poder trabajar con los datos pues nuestra capa de alcaldías. calles tienen un SRC distinto.
for i in range(len(shuffled_data)):
    
    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)

    target = osr.SpatialReference()
    target.ImportFromEPSG(32614)

    transform = osr.CoordinateTransformation(source, target)

    point = ogr.CreateGeometryFromWkt("POINT ({} {})".format(shuffled_data['latitud_destino'][i], shuffled_data['longitud_destino'][i]))
    point.Transform(transform)
    
    point2 = ogr.CreateGeometryFromWkt("POINT ({} {})".format(shuffled_data['latitud_origen'][i], shuffled_data['longitud_origen'][i]))
    point2.Transform(transform)
    
    shuffled_data['latitud_destino'][i] , shuffled_data['longitud_destino'][i]  = point.GetX(), point.GetY()
    shuffled_data['latitud_origen'][i] , shuffled_data['longitud_origen'][i]  = point2.GetX(), point2.GetY()
    
# Creamos un par de listas con los orígenes y destinos de nuestros datos

origen_lat_long = []
destino_lat_long = []

b = shuffled_data['latitud_origen'].tolist()
c = shuffled_data['longitud_origen'].tolist()

for i in range(len(shuffled_data)):
    origen_lat_long.append([b[i], c[i]])



d = shuffled_data['latitud_destino'].tolist()
g = shuffled_data['longitud_destino'].tolist()

for i in range(len(shuffled_data)):
    destino_lat_long.append([d[i], g[i]])
    
# Y una última lista para guardar las rutas obtenidas.

rutas = []


'''Dijkstra'''




iterator = 0

for i in range(len(shuffled_data)):
    
    path_to_file_cdmx = '/Volumes/Escritorio_Externo/Users/raulguzman/Documents/info/Tesis_maestria/RAUL TESIS DATOS/DATOS FINALES/calles/callesCDMX.shp'  


    vectorLayer = QgsVectorLayer(path_to_file_cdmx, 'lines')
    director = QgsVectorLayerDirector(vectorLayer, 3, 'yes', '1', 'yes', QgsVectorLayerDirector.DirectionBoth)
    strategy = QgsNetworkDistanceStrategy()
    director.addStrategy(strategy)

    builder = QgsGraphBuilder(vectorLayer.sourceCrs())


    startPoint = QgsPointXY(origen_lat_long[i][0], origen_lat_long[i][1])
    endPoint = QgsPointXY(destino_lat_long[i][0], destino_lat_long[i][1])

    tiedPoints = director.makeGraph(builder, [startPoint, endPoint])
    tStart, tStop = tiedPoints

    graph = builder.graph()
    idxStart = graph.findVertex(tStart)
    idxEnd = graph.findVertex(tStop)

    (tree, costs) = QgsGraphAnalyzer.dijkstra(graph, idxStart, 0)

    if tree[idxEnd] == -1:
        #raise Exception('No route!')
        continue # dejamos que continue porque hay ocasiones en las que la ruta no existe, por lo que debemos quitar esos puntos de origen y destino

    # Total cost
    cost = costs[idxEnd]

    # Add last point
    route = [graph.vertex(idxEnd).point()]

    # Iterate the graph
    while idxEnd != idxStart:
        idxEnd = graph.edge(tree[idxEnd]).fromVertex()
        route.insert(0, graph.vertex(idxEnd).point())
        
    rutas.append(route)
    iterator += 1 
    
print(iterator)    
    
'''comprobamos cuántas rutas generó nuestro algoritmo recordando que no siempre existe una ruta. Si obtenemos un número menor al de las coordenadas de inicio y término,
sólo comparamos las rutas creadas con sus respectivas coordenadas de inicio y desechamos aquellas que no hayan podido crear un ruta'''

print(len(rutas))

# como QGIS nos devuelve un objeto, necesitamos convertir cada punto de la ruta en algo que sea más sencillo de accesar
resultado = []

for j in range(len(rutas)):
    
    a = []

    for i in range(len(rutas[j])):

        point = ogr.CreateGeometryFromWkt("POINT ({} {})".format(rutas[j][i][0], rutas[j][i][1]))
        a.append((point.GetX(), point.GetY()))
    
    resultado.append(a)

# Finalmente agregamos las rutas obtenidas al dataset inicial

shuffled_data['Polyline'] = resultado

# determining the name of the file
file_name = 'datos_generados_encuesta.csv'
  
# saving the excel
shuffled_data.to_csv(file_name)
print('DataFrame is written to Excel File successfully.')

end = time.time()
print(f"\nTotal setup time {end - start:.2f} s")


